In [7]:
import sys
import pandas as pd
import os
from matplotlib import pyplot as plt
from scipy.stats import hmean
import tikzplotlib

dir_latency = "./paper/gzhm_variants_latency"
dir_thrput = "./paper/gzhm_external"
variants = next(os.walk(dir_latency))[1]
print(variants)

['TRHM', 'GZHM_DELETE', 'RHM', 'GZHM', 'GRHM']


In [10]:
def humanize_nanoseconds(sec):
    time_str = ('%.2f ns') % (sec)
    if (sec > 1000):
        time_str = ('%.2f us') % (sec / 1000.0)
    if (sec > 1000_000):
        time_str = ('%.2f ms') % (sec / 1000_000.0)
    return time_str

def latency_distribution(dir, ops):
    summaries = pd.DataFrame()
    all_summaries = pd.DataFrame()
    for op in ops:
        for d in variants:
            df = pd.read_csv('./%s/%s/churn_latency.txt' % (dir, d), delim_whitespace=True)
            df = df.loc[(df["op"]==op)]
            if (len(df)==0):
                continue
            summaries[f'{d}'] = df['latency'].describe(percentiles=[.50, .90, .99, .9999])
        print(op)
        hsum = pd.DataFrame()
        for column in summaries.columns:
            hsum[column] = (summaries[column].map(lambda x : humanize_nanoseconds(x)))
        print(hsum.to_latex())
latency_distribution(dir_latency, ["DELETE", "INSERT", "LOOKUP"])

DELETE
\begin{tabular}{llllll}
\toprule
 & TRHM & GZHM_DELETE & RHM & GZHM & GRHM \\
\midrule
count & 1.32 ms & 1.32 ms & 1.32 ms & 1.32 ms & 1.32 ms \\
mean & 10.49 us & 32.45 us & 109.56 us & 10.12 us & 9.98 us \\
std & 1.31 us & 8.57 us & 26.90 us & 1.20 us & 1.12 us \\
min & 8.01 us & 14.45 us & 32.88 us & 7.85 us & 7.67 us \\
50% & 10.34 us & 31.14 us & 106.57 us & 10.02 us & 9.88 us \\
90% & 11.53 us & 43.41 us & 145.08 us & 11.04 us & 10.90 us \\
99% & 13.80 us & 58.80 us & 185.10 us & 12.65 us & 12.50 us \\
99.99% & 28.66 us & 95.86 us & 330.76 us & 20.10 us & 18.95 us \\
max & 281.55 us & 318.22 us & 477.60 us & 277.93 us & 266.06 us \\
\bottomrule
\end{tabular}

INSERT
\begin{tabular}{llllll}
\toprule
 & TRHM & GZHM_DELETE & RHM & GZHM & GRHM \\
\midrule
count & 1.32 ms & 1.32 ms & 1.32 ms & 1.32 ms & 1.32 ms \\
mean & 17.16 us & 16.32 us & 38.88 us & 49.36 us & 58.24 us \\
std & 1.73 us & 1.69 us & 4.64 us & 4.56 us & 7.84 ms \\
min & 12.21 us & 12.17 us & 21.26 us & 35.53 u

In [1]:
def humanize_bytes(bytes):
    bytes_str = ('%d B') % (bytes)
    if (bytes > 1024):
        bytes_str = ('%d KB') % (bytes / (1024))
    if (bytes > 1024 * 1024):
        bytes_str = ('%d MB') % (bytes / (1024* 1024))
    if (bytes > 1024 * 1024 * 1024):
        bytes_str = ('%d GB') % (bytes / (1024* 1024 * 1024))
    return bytes_str

def memory_usage(dir):    
    data = []
    labels = []
    df = pd.DataFrame()
    for d in variants:
        f_variant = open("%s/%s/test_params.txt" % (dir, d), "r")
        lines = f_variant.readlines()
        memory_usage = int(lines[0])
        data.append(memory_usage)
        labels.append(d)
    df['Hashmap'] = labels
    df['USAGE'] = data
    df['Size'] = df['USAGE'].map(lambda x: humanize_bytes(x))
    df['Space Efficiency'] = df['USAGE'].map(lambda x: (100 * x) / (0.95 * (2**22) * 16))
    print(df[['Hashmap', "Size", "Space Efficiency"]].to_latex(float_format="%.2f"))
memory_usage(dir_thrput)

NameError: name 'dir_thrput' is not defined